In [1]:
import tensorflow as tf
import keras_tuner as kt
import sys
sys.path.append("..")
import tools
%load_ext autoreload
%autoreload 2

2024-04-29 10:54:50.173119: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 10:54:50.173195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 10:54:50.174925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 10:54:50.187575: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 10:54:52.243154: W tensorflow/compiler/tf2

In [2]:
dataset_train = tf.data.TFRecordDataset(["../DATA/train1.tfrecord"])
tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_train)
dataset_train = dataset_train.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
dataset_train = dataset_train.shuffle(5 * 128).batch(128).prefetch(2)
dataset_val = tf.data.TFRecordDataset(["../DATA/test1.tfrecord"])
dataset_val = dataset_val.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
dataset_val = dataset_val.batch(128).prefetch(2)
strategy = tf.distribute.get_strategy()
FE = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=0.95)
CE = tf.keras.losses.BinaryCrossentropy()
earlystopping_kb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5 * 6,
                                                    verbose=1,
                                                    restore_best_weights=True)
terminateonnan_kb = tf.keras.callbacks.TerminateOnNaN()
reducelronplateau_kb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95,
                                                            patience=6, verbose=1)
tuner = kt.Hyperband(hypermodel=tools.hypertuneModels.StockHyperModel(tfrecord_shape, tools.model.custom_loss_sum([FE, CE])),
                    objective=kt.Objective('val_f1_score', "max"),
                    max_epochs=64,
                    factor=int(4),
                    hyperband_iterations=64,
                    directory="/astro/users/kmrakovc/Projects/Tuner",
                    overwrite=False,
                    project_name="AsteroidsNN_Tuner",
                    distribution_strategy=strategy)

2024-04-29 10:54:59.112187: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Reloading Tuner from /astro/users/kmrakovc/Projects/Tuner/AsteroidsNN_Tuner/tuner0.json


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
best_hps = tools.hypertuneModels.get_best_hyperparameters(tuner, num_trials=10)

In [ ]:
best_hps